In [25]:
import os
from dotenv import load_dotenv
from mistralai import Mistral
from instructor import from_mistral, Mode

load_dotenv()

client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

instructor_client = from_mistral(
    client=client,
    model="mistral-large-latest",
    mode=Mode.MISTRAL_TOOLS,
    use_async=False,
)


In [ ]:
from pydantic import BaseModel, Field


class ListaTopicos(BaseModel):
    topics: list[str] = Field(
        ..., title="Uma lista de tópicos com base na mensagem recebida."
    )
    topicos_associados: list[str] = Field(
        ..., title="Uma lista de tópicos associados com base na mensagem recebida."
    )


system_prompt = """Você é um especialista em gerar tópicos de conversa.
Para cada mensagem recebida, gere uma lista contendo tópicos relacionados ao texto. Por exemplo,
se o usuário disser "Eu preciso de uma lista de tópicos para falar sobre comunicação política",
considere possíveis temas, autores, teorias, conceitos e exemplos que possam ser relevantes e use-os para gerar a lista.
"""

user_prompt = (
    "Gere uma lista de tópicos sobre comunicação política e teoria da comunicação."
)

resp = instructor_client.messages.create(
    response_model=ListaTopicos,
    messages=[
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    temperature=1,
)


In [23]:
print(resp)


topics=['Comunicação Política', 'Teoria da Comunicação', 'Estratégias de Comunicação Política', 'Propaganda Política', 'Media e Política', 'Discurso Político', 'Marketing Político', 'Redes Sociais e Política', 'Comunicação e Democracia', 'Teorias da Persuasão', 'Análise de Discurso', 'Comunicação e Poder', 'Mídia Digital e Política', 'Campanhas Eleitorais', 'Opinião Pública', 'Comunicação Institucional', 'Retórica Política', 'Efeitos da Mídia', 'Comunicação e Cidadania', 'Políticas Públicas de Comunicação', 'Jornalismo Político']


In [19]:
class Tweet(BaseModel):
    text: str = Field(
        ...,
        title="Um tweet gerado com base nos tópicos fornecidos e nas regras estabelecidas.",
    )


system_prompt = """Você é um especialista em gerar tweets.
Você receberá um tópico, com base nele, deverá gerar um tweet. Para isto, siga as seguintes regras:
1. Certifique-se de que o tweet seja relevante, informativo e opinativo. 
2. Você é livre para imaginar cenários, exemplos e detalhes adicionais para enriquecer os tweets.
3. Para garantir a variedade, evite repetir ideias semelhantes nos tweets.
4. Seus tweets devem variar em estilo e conteúdo, abordando diferentes aspectos dos tópicos fornecidos.
5. Varie o sentimento (positivo, negativo, neutro) dos tweets para manter o interesse dos leitores.
6. Varie a emoção (alegria, tristeza, raiva, medo, surpresa, nojo) dos tweets para manter o interesse dos leitores.
8. Não use emojis ou menções a usuários do Twitter.
"""

tweets = []

for topic in resp.topics:
    tweet = instructor_client.messages.create(
        response_model=Tweet,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": topic,
            },
        ],
        temperature=1,
    )

    tweets.append(tweet.text)


In [ ]:
print(tweets)


['A comunicação política na era digital tem transformado a forma como os líderes se conectam com seus eleitores. Plataformas como Twitter e Facebook se tornaram campos de batalha ideológica, onde um único post pode viralizar e moldar opiniões em questão de minutos. Isso exige que os políticos sejam mais transparentes e acessíveis, mas também aumenta o risco de desinformação e polarização. É um desafio constante, mas essencial para a democracia moderna.', 'A teoria da comunicação nos mostra como a mensagem pode ser distorcida em cada etapa do processo. Desde a codificação até a decodificação, cada passo é crucial para garantir que o receptor compreenda a mensagem como o emissor pretendia. #ComunicaçãoEficaz', 'Estratégias de Comunicação Política: A comunicação política nunca foi tão desafiadora. Com a proliferação das fake news, é crucial que os políticos adotem estratégias transparentes e éticas. A autenticidade e a proximidade com o eleitor são fundamentais para reconquistar a confian

In [36]:
from instructor import llm_validator
from pydantic import BeforeValidator, ValidationError
from typing_extensions import Annotated


class QuestionAnswer(BaseModel):
    question: str
    answer: Annotated[
        str,
        BeforeValidator(
            llm_validator(
                "don't say objectionable things",
                client=instructor_client,
                model="mistral-large-latest",
            )
        ),
    ]


try:
    qa = QuestionAnswer(
        question="What is the meaning of life?",
        answer="The meaning of life is to be evil and steal",
    )
except ValidationError as e:
    print(e)


1 validation error for QuestionAnswer
answer
  Assertion failed, The statement promotes harmful behavior, which is not acceptable. [type=assertion_error, input_value='The meaning of life is to be evil and steal', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/assertion_error
